In [1]:
import pandas as pd
import numpy as np
import re
import math
from tqdm import tqdm
tqdm.pandas()

In [2]:
df = pd.read_csv('/home/intern2/fire_dev/Final_fire_news_4.csv',encoding='utf-8-sig')

In [3]:
df_cluster = pd.read_csv('/home/intern2/fire_dev/JY_WORKPLACE/dsfkhjdsl.csv',encoding='utf-8-sig')

In [4]:
df_cluster

,_id,Title,Body,li
0,AKR20221231041400061,신분당선 판교∼정자역 사이서 전동차 멈춰1시간가량 불편,이영주 기자 = 31일 오후 9시 40분께 경기도 성남시 신분당선 판교역과 정자역 ...,NaN
1,AKR20221231035000099,남아공 가스트럭 폭발사고 사망자 34명으로 늘어,김성진 특파원 = 남아프리카공화국에서 지난 크리스마스이브에 발생한 가스 트럭 폭발 ...,NaN
2,AKR20221231025151003,"선로에 떨어진 부직포 때문에SRT, 운행 차질에 차량 고장까지",전날 선로 전기공급 차단 사고 여파이틀째 운행 취소 연기 잇따라 잇단 철도사고에 국...,NaN
3,AKR20221231025151003,"선로에 떨어진 부직포 때문에SRT, 운행 차질에 차량 고장까지",전날 선로 전기공급 차단 사고 여파이틀째 운행 취소 연기 잇따라 잇단 철도사고에 국...,NaN
4,AKR20221231025100003,선로 전기공급 끊기고 SRT 고장이틀째 대거 운행지연 취소,첫날 KTX SRT 최대 2시간여 운행지연열차 운행 축소 터널 하자보수 공사때 쓴 ...,NaN
...,...,...,...,...
14052,AKR20220607103300530,"소방청 가스안전공사, 신에너지 사고 대응 업무협약",계승현 기자 = 소방청은 8일 한국가스안전공사와 수소 등 신에너지 관련 사고에 대응...,NaN
14053,PYH20220608015200017,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축",LG유플러스가 서울 지하철 8호선 18개 역사에 '스마트스테이션'을 구축하는 사업을...,NaN
14054,AKR20220607168000017,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축 완료",역사 내 사고에 신속 대응해 역무 효율성 높여 임성호 기자 = LG유플러스는 서울 ...,NaN
14055,AKR20220608017300530,제조업 운반 하역 작업서 부상 가장 많이 발생전체 19.3%,김승욱 기자 = 3일 이상의 휴업이 필요한 부상 사고의 20% 가까이가 제조업 운반...,NaN


In [5]:
df.columns

Index(['_id', 'send_timestamp', 'Title', 'Body',
       'YNewsML.Metadata.Region.RegionCode', 'Summarization', 'NamedEntity',
       'Keyword', 'img_link', 'issue_word', 'reason', 'class_code',
       'class_name'],
      dtype='object')

In [6]:
df_news = df[['_id','Title','Body','issue_word','reason']]

In [7]:
df_news = df_news[df_news['issue_word'] != np.nan].dropna(subset='issue_word').reset_index().drop('index',axis=1)

In [8]:
df_news = df_news.drop_duplicates(subset='_id').reset_index().drop('index',axis=1)

In [9]:
from kiwipiepy import Kiwi
kiwi = Kiwi()
b = pd.DataFrame()

for i in df_news['Body']:
    a = []
    c = ''
    for result, score in kiwi.analyze(i, top_n=1):
        for j in range(len(result)):
            if result[j][1] == 'NNG':
                c += result[j][0] + ', '
    a.append(c)
    b = pd.concat([b,pd.DataFrame([a])],axis=0)

In [10]:
b = b.reset_index().drop('index',axis=1)

In [11]:
b

,0
0,"특파원, 발생, 북서부, 카지노, 호텔, 대, 화재, 사망자, 최소, 통신, 외신,..."
1,"감정, 결과, 여성, 남성, 영주, 기자, 신원, 사고, 발생, 이틀, 확인, 남부..."
2,"보람, 기자, 서울시장, 새해, 시민, 안전, 도시, 환경, 말, 시장, 발표, 신..."
3,"계승, 현, 기자, 다음, 주, 금요일, 오후, 수도, 비, 눈, 곳, 기상청, 아..."
4,"환, 기자, 목숨, 방음, 터널, 화재, 관련, 방음, 터널, 설치, 긴급, 안전,..."
...,...
3377,"변호사, 사무실, 빌딩, 화재, 경찰, 방화, 여부, 조사,"
3378,"변호사, 사무실, 빌딩, 화재, 소방, 사망, 추정,"
3379,"선풍기, 시간, 설정, 과열, 방지, 옷, 기자, 여름, 예년, 전망, 가운데, 에..."
3380,"문, 영, 기자, 원자력, 안전, 위원회, 원자력, 안전, 정보, 공개, 소통, 법..."


In [12]:
df_news = pd.concat([df_news,b],axis=1).drop_duplicates(subset='_id').reset_index().drop('index',axis=1)
df_news

,_id,Title,Body,issue_word,reason,0
0,AKR20221231022000076,캄보디아 카지노 호텔 화재 사망자 최소 27명으로수색 종료,강종훈 특파원 = 지난 28일 발생한 캄보디아 북서부 카지노 호텔 대형 화재로 인한...,화재,미상,"특파원, 발생, 북서부, 카지노, 호텔, 대, 화재, 사망자, 최소, 통신, 외신,..."
1,AKR20221231022600061,과천 방음터널 화재 사망자 5명 신원 확인모녀도 참변,DNA 감정 결과 여성 3명 남성 2명60대 3명 30대 1명 20대 1명 이영주 ...,화재,NaN,"감정, 결과, 여성, 남성, 영주, 기자, 신원, 사고, 발생, 이틀, 확인, 남부..."
2,AKR20221230139200004,오세훈 안전에 모든 노력동행 매력도시 도약,윤보람 기자 = 오세훈 서울시장은 2023년 새해를 맞아 서울시민이 언제 어디를 가...,이태원압사참사,NaN,"보람, 기자, 서울시장, 새해, 시민, 안전, 도시, 환경, 말, 시장, 발표, 신..."
3,AKR20221231016500530,금요일 오후 수도권 강원 영서에 비나 눈,계승현 기자 = 다음 주 금요일 오후에는 수도권과 강원영서에 비 또는 눈이 오는 곳...,산불,미상,"계승, 현, 기자, 다음, 주, 금요일, 오후, 수도, 비, 눈, 곳, 기상청, 아..."
4,AKR20221230129100063,대전 세종 충남 국도 내 방음터널 8곳세종시 긴급 안전 점검,이주형 강수환 기자 = 5명의 목숨을 앗아간 경기 과천 제2경인고속도로 방음터널 화...,화재,NaN,"환, 기자, 목숨, 방음, 터널, 화재, 관련, 방음, 터널, 설치, 긴급, 안전,..."
...,...,...,...,...,...,...
3377,AKR20220609084700053,대구 변호사 사무실 빌딩 화재경찰 방화 여부 조사,대구 변호사 사무실 빌딩 화재경찰 방화 여부 조사,화재,NaN,"변호사, 사무실, 빌딩, 화재, 경찰, 방화, 여부, 조사,"
3378,AKR20220609084300053,대구 변호사 사무실 빌딩 화재소방 7명 사망 추정,대구 변호사 사무실 빌딩 화재소방 7명 사망 추정,소방,NaN,"변호사, 사무실, 빌딩, 화재, 소방, 사망, 추정,"
3379,AKR20220609058700530,에어컨 등 냉방기 화재로 5년간 12명 사망단독 콘센트 사용,선풍기는 시간설정으로 과열 방지옷 널어두지 않도록 김윤구 기자 = 올여름은 예년보다...,화재,전기적요인,"선풍기, 시간, 설정, 과열, 방지, 옷, 기자, 여름, 예년, 전망, 가운데, 에..."
3380,AKR20220608090500017,"오늘부터 원자력안전소통법 시행원안위, 정보공유센터 구축",문다영 기자 = 원자력안전위원회는 9일 '원자력안전 정보공개 및 소통에 관한 법률'...,대한민국,NaN,"문, 영, 기자, 원자력, 안전, 위원회, 원자력, 안전, 정보, 공개, 소통, 법..."


In [13]:
df2 = pd.DataFrame(df_news.groupby('issue_word').sum()[0]).reset_index().rename(columns={0:'NNG'}).drop(2)

In [14]:
df2['issue_word'].tolist()

['10',
 '10.29 참사',
 '2022 쉐이크아웃 코리아',
 '2차전',
 '3대',
 '4.1 지진',
 '4.9',
 '4·16',
 '5·18',
 '88올림픽',
 'Bon Om Touk',
 'CES',
 'COP26',
 'DOUG',
 'DXKOREA',
 'GSTAR)',
 'INFOSAN) 국제회의',
 'KT',
 'MDSAP',
 'MMA',
 'SPL사고',
 'SRF설명회',
 'SSG랜더스',
 'SpaceNet8) 챌린지',
 'Steelie Awards',
 'Vigilant Storm',
 '가뭄',
 '가습기살균제사건',
 '가요콘서트',
 '가을문화축제',
 '거리응원전',
 '건강친화기업 인증제도 성과대회',
 '건설기술 심포지엄',
 '경기도생활대축전',
 '경주',
 '고성산불 사건',
 '고인돌전국마라톤대회',
 '공격편대군 훈련',
 '공주깍두기축제',
 '광화문광장 마켓',
 '교통문화발전대회',
 '교통사고 줄이기 한마음대회',
 '구의역',
 '국가손상조사감시사업',
 '국민생활과학기술포럼',
 '국민안전 발명챌린지',
 '국정감사',
 '국제가전제품박람회',
 '국제안전보건전시회',
 '국화향 가득한 가을밤 콘서트',
 '금융',
 '금융시장 안전성 점검',
 '급성심장정지조사 심포지엄',
 '긴급구조종합훈련',
 '남양주 개물림 사망사건',
 '남이장군 사당제',
 '논술',
 '대테러종합훈련',
 '대한민국',
 '대한민국 범죄예방대상',
 '대한민국 안전산업박람회',
 '더 스토리(The story) 인간문화재 신영희',
 '동일본대지진',
 '동작가족 건강체험관',
 '동장군 축제',
 '디왈리',
 '디왈리 축제',
 '런던협약',
 '레고랜드사태',
 '로이 끄라통 축제',
 '막국수닭갈비축제',
 '무역산업포럼;',
 '민방위 훈련',
 '방사성폐기물안전관리국제심포지엄',
 '밸프스 플러스 화학 안전 캠페인',
 '범도민 안전체험한마당',
 '보행안전 국제 세미나',
 '봉화',
 '봉화

In [15]:
df1 = df[['Title','Body','send_timestamp']].reset_index().drop('index',axis=1)

In [16]:
df1

,Title,Body,send_timestamp
0,신분당선 판교∼정자역 사이서 전동차 멈춰1시간가량 불편,이영주 기자 = 31일 오후 9시 40분께 경기도 성남시 신분당선 판교역과 정자역 ...,2022-12-31T23:20:59
1,남아공 가스트럭 폭발사고 사망자 34명으로 늘어,김성진 특파원 = 남아프리카공화국에서 지난 크리스마스이브에 발생한 가스 트럭 폭발 ...,2022-12-31T18:08:13
2,"선로에 떨어진 부직포 때문에SRT, 운행 차질에 차량 고장까지",전날 선로 전기공급 차단 사고 여파이틀째 운행 취소 연기 잇따라 잇단 철도사고에 국...,2022-12-31T16:16:20
3,"선로에 떨어진 부직포 때문에SRT, 운행 차질에 차량 고장까지",전날 선로 전기공급 차단 사고 여파이틀째 운행 취소 연기 잇따라 잇단 철도사고에 국...,2022-12-31T16:16:20
4,선로 전기공급 끊기고 SRT 고장이틀째 대거 운행지연 취소,첫날 KTX SRT 최대 2시간여 운행지연열차 운행 축소 터널 하자보수 공사때 쓴 ...,2022-12-31T12:57:34
...,...,...,...
14052,"소방청 가스안전공사, 신에너지 사고 대응 업무협약",계승현 기자 = 소방청은 8일 한국가스안전공사와 수소 등 신에너지 관련 사고에 대응...,2022-06-08T10:00:07
14053,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축",LG유플러스가 서울 지하철 8호선 18개 역사에 '스마트스테이션'을 구축하는 사업을...,2022-06-08T09:20:49
14054,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축 완료",역사 내 사고에 신속 대응해 역무 효율성 높여 임성호 기자 = LG유플러스는 서울 ...,2022-06-08T09:00:02
14055,제조업 운반 하역 작업서 부상 가장 많이 발생전체 19.3%,김승욱 기자 = 3일 이상의 휴업이 필요한 부상 사고의 20% 가까이가 제조업 운반...,2022-06-08T09:00:00


In [17]:
df1 = df1.drop_duplicates('Body').reset_index().drop('index',axis=1)

In [18]:
df2 = df2.reset_index().drop('index',axis=1)

In [19]:
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return float(len(s1.intersection(s2)) / len(s1.union(s2)))

In [20]:
jac =[]
for i in range(len(df2)):
        for j in range(len(df2)):
                jac.append([df2['issue_word'][i],df2['issue_word'][j],jaccard_similarity(df2['NNG'][i],df2['NNG'][j])])

In [21]:
cond1 = pd.DataFrame(jac)[2] > 0.7
cond2 = 1 > pd.DataFrame(jac)[2]

In [22]:
pd.DataFrame(jac)[cond1&cond2]

,0,1,2
24535,세월호침몰사고,핼러윈,0.715536
34451,이태원압사참사,화재,0.720195
41299,지진,힌남노태풍,0.707276
47195,태풍,화재,0.702997
47199,태풍,힌남노태풍,0.759317
48823,통신,통신4사고,0.704225
49058,통신4사고,통신,0.704225
53675,핼러윈,세월호침몰사고,0.715536
54661,화재,이태원압사참사,0.720195
54715,화재,태풍,0.702997


In [24]:
df_cluster

,_id,Title,Body,li
0,AKR20221231041400061,신분당선 판교∼정자역 사이서 전동차 멈춰1시간가량 불편,이영주 기자 = 31일 오후 9시 40분께 경기도 성남시 신분당선 판교역과 정자역 ...,NaN
1,AKR20221231035000099,남아공 가스트럭 폭발사고 사망자 34명으로 늘어,김성진 특파원 = 남아프리카공화국에서 지난 크리스마스이브에 발생한 가스 트럭 폭발 ...,NaN
2,AKR20221231025151003,"선로에 떨어진 부직포 때문에SRT, 운행 차질에 차량 고장까지",전날 선로 전기공급 차단 사고 여파이틀째 운행 취소 연기 잇따라 잇단 철도사고에 국...,NaN
3,AKR20221231025151003,"선로에 떨어진 부직포 때문에SRT, 운행 차질에 차량 고장까지",전날 선로 전기공급 차단 사고 여파이틀째 운행 취소 연기 잇따라 잇단 철도사고에 국...,NaN
4,AKR20221231025100003,선로 전기공급 끊기고 SRT 고장이틀째 대거 운행지연 취소,첫날 KTX SRT 최대 2시간여 운행지연열차 운행 축소 터널 하자보수 공사때 쓴 ...,NaN
...,...,...,...,...
14052,AKR20220607103300530,"소방청 가스안전공사, 신에너지 사고 대응 업무협약",계승현 기자 = 소방청은 8일 한국가스안전공사와 수소 등 신에너지 관련 사고에 대응...,NaN
14053,PYH20220608015200017,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축",LG유플러스가 서울 지하철 8호선 18개 역사에 '스마트스테이션'을 구축하는 사업을...,NaN
14054,AKR20220607168000017,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축 완료",역사 내 사고에 신속 대응해 역무 효율성 높여 임성호 기자 = LG유플러스는 서울 ...,NaN
14055,AKR20220608017300530,제조업 운반 하역 작업서 부상 가장 많이 발생전체 19.3%,김승욱 기자 = 3일 이상의 휴업이 필요한 부상 사고의 20% 가까이가 제조업 운반...,NaN


In [25]:
from kiwipiepy import Kiwi
kiwi = Kiwi()
b = pd.DataFrame()

for i in df_cluster['Body']:
    a = []
    c = ''
    for result, score in kiwi.analyze(i, top_n=1):
        for j in range(len(result)):
            if result[j][1] == 'NNG':
                c += result[j][0] + ', '
    a.append(c)
    b = pd.concat([b,pd.DataFrame([a])],axis=0)

In [26]:
b = b.reset_index().drop('index',axis=1).rename(columns={0:'NNG'})

In [27]:
df_cluster = pd.concat([df_cluster,b],axis=1)

In [28]:
df_cluster.dropna()

,_id,Title,Body,li,NNG
92,AKR20221229103454061,과천 제2경인고속도 방음터널서 큰 불5명 사망 37명 부상,사망자들 모두 주변 지나던 승용차서 발견터널 내 차량 45대 소실 트럭서 시작된 불...,경인고속도 화재,"사망자, 모두, 주변, 승용차, 발견, 터널, 차량, 소실, 트럭, 시작, 불, 플..."
112,AKR20221229103453061,과천 제2경인고속도 방음터널서 화재5명 사망 37명 부상,버스 트럭 추돌사고로 발생한 불이 터널로 확산2시간 만에 완진 목격자 대부분 운전자...,경인고속도 화재,"버스, 트럭, 추돌, 사고, 발생, 불, 터널, 확산, 와, 목격자, 대부분, 운전..."
545,AKR20221223016853004,지하철 3호선 화재로 운행중단한파속 출근길 대란,약수∼구파발 양방향 1시간45분 멈춰고압 케이블 원인 추정 고현실 오보람 기자 = ...,3호선 화재,"약수, 양방향, 고압, 케이블, 원인, 추정, 현실, 보람, 기자, 오전, 지하철,..."
2890,AKR20221109084152530,소방청 용산소방서장 지휘 상황관리 적극 관여,행안부 형식적 대응으로 빠른 보고 못 했다개선할 것 야당 '희생자 신상공개' 요구엔...,이태원 참사,"행안부, 형식, 대응, 보고, 개선, 야당, 희생자, 신상, 공개, 요구, 검토, ..."
2891,AKR20221109084152530,소방청 용산소방서장 지휘 상황관리 적극 관여,행안부 형식적 대응으로 빠른 보고 못 했다개선할 것 야당 '희생자 신상공개' 요구엔...,이태원 참사,"행안부, 형식, 대응, 보고, 개선, 야당, 희생자, 신상, 공개, 요구, 검토, ..."
...,...,...,...,...,...
11736,AKR20220808090853065,246mm 폭우에 인천 피해 500건 육박,5개 구에 산사태 예보도로 재래시장도 빗물에 잠겨 손현규 최은지 기자 = 8일 인천...,중부 집중호우,"구, 산사태, 예보, 도로, 재래시장, 빗물, 규, 지, 기자, 시간, 폭우, 도로..."
11756,AKR20220808114552061,중부지방에 쏟아진 폭우로 곳곳서 도로통제 침수피해,서울남부 인천 저녁에 시간당 50∼90㎜ 한강 곳곳 홍수주의 충주댐 2년만 방류 8...,중부 집중호우,"남부, 저녁, 시간, 곳곳, 방류, 수도, 중부, 지방, 중심, 폭우, 저녁, 남부..."
11884,AKR20220805105700061,이천 4층 빌딩 화재로 5명 사망 44명 부상인명 수색 종료,사망자 60∼80대 환자 및 50대 간호사고령 투석으로 피해 커진 듯 발화 지점 3...,이천 4층빌딩 화재,"사망자, 환자, 간호사, 고령, 투석, 발화, 지점, 층, 장, 철거, 작업, 진행..."
13102,AKR20220701074553004,종로 르메이에르빌딩 5분간 흔들1천여명 한때 대피,옥상 냉각타워 파손 영향날개 부러진 대형 팬이 회전하면서 균형 잃어 진동 4시간 가...,종로 르미에르빌딩,"옥상, 냉각, 타워, 파손, 영향, 날개, 대형, 팬, 회전, 균형, 진동, 전면,..."


In [29]:
df_cluster.dropna()['li'] == '경인고속도 화재'

92        True
112       True
545      False
2890     False
2891     False
         ...  
11736    False
11756    False
11884    False
13102    False
13575    False
Name: li, Length: 76, dtype: bool

In [30]:
df_cluster_KI = df_cluster[df_cluster['li'] == '경인고속도 화재'].reset_index().drop('index',axis=1)

In [31]:
df_cluster_KI

,_id,Title,Body,li,NNG
0,AKR20221229103454061,과천 제2경인고속도 방음터널서 큰 불5명 사망 37명 부상,사망자들 모두 주변 지나던 승용차서 발견터널 내 차량 45대 소실 트럭서 시작된 불...,경인고속도 화재,"사망자, 모두, 주변, 승용차, 발견, 터널, 차량, 소실, 트럭, 시작, 불, 플..."
1,AKR20221229103453061,과천 제2경인고속도 방음터널서 화재5명 사망 37명 부상,버스 트럭 추돌사고로 발생한 불이 터널로 확산2시간 만에 완진 목격자 대부분 운전자...,경인고속도 화재,"버스, 트럭, 추돌, 사고, 발생, 불, 터널, 확산, 와, 목격자, 대부분, 운전..."


In [32]:
jac =[]
for i in range(len(df_cluster_KI)):
        for j in range(len(df_cluster_KI)):
                jac.append([df_cluster_KI['Body'][i],df_cluster_KI['Body'][j],jaccard_similarity(df_cluster_KI['NNG'][i],df_cluster_KI['NNG'][j])])

In [33]:
cond1 = 1 > pd.DataFrame(jac)[2] 

In [34]:
pd.DataFrame(jac).loc[cond1]

,0,1,2
1,사망자들 모두 주변 지나던 승용차서 발견터널 내 차량 45대 소실 트럭서 시작된 불...,버스 트럭 추돌사고로 발생한 불이 터널로 확산2시간 만에 완진 목격자 대부분 운전자...,0.634259
2,버스 트럭 추돌사고로 발생한 불이 터널로 확산2시간 만에 완진 목격자 대부분 운전자...,사망자들 모두 주변 지나던 승용차서 발견터널 내 차량 45대 소실 트럭서 시작된 불...,0.634259


In [35]:
df_cluster_KI

,_id,Title,Body,li,NNG
0,AKR20221229103454061,과천 제2경인고속도 방음터널서 큰 불5명 사망 37명 부상,사망자들 모두 주변 지나던 승용차서 발견터널 내 차량 45대 소실 트럭서 시작된 불...,경인고속도 화재,"사망자, 모두, 주변, 승용차, 발견, 터널, 차량, 소실, 트럭, 시작, 불, 플..."
1,AKR20221229103453061,과천 제2경인고속도 방음터널서 화재5명 사망 37명 부상,버스 트럭 추돌사고로 발생한 불이 터널로 확산2시간 만에 완진 목격자 대부분 운전자...,경인고속도 화재,"버스, 트럭, 추돌, 사고, 발생, 불, 터널, 확산, 와, 목격자, 대부분, 운전..."


In [36]:
df_cluster

,_id,Title,Body,li,NNG
0,AKR20221231041400061,신분당선 판교∼정자역 사이서 전동차 멈춰1시간가량 불편,이영주 기자 = 31일 오후 9시 40분께 경기도 성남시 신분당선 판교역과 정자역 ...,NaN,"기자, 오후, 사이, 선로, 전동차, 고장, 열차, 승강장, 선로, 승객, 시간, ..."
1,AKR20221231035000099,남아공 가스트럭 폭발사고 사망자 34명으로 늘어,김성진 특파원 = 남아프리카공화국에서 지난 크리스마스이브에 발생한 가스 트럭 폭발 ...,NaN,"특파원, 크리스마스, 이브, 발생, 가스, 트럭, 폭발, 사고, 관련, 사망자, 보..."
2,AKR20221231025151003,"선로에 떨어진 부직포 때문에SRT, 운행 차질에 차량 고장까지",전날 선로 전기공급 차단 사고 여파이틀째 운행 취소 연기 잇따라 잇단 철도사고에 국...,NaN,"전날, 선로, 전기, 공급, 차단, 사고, 여파, 이틀, 운행, 취소, 연기, 철도..."
3,AKR20221231025151003,"선로에 떨어진 부직포 때문에SRT, 운행 차질에 차량 고장까지",전날 선로 전기공급 차단 사고 여파이틀째 운행 취소 연기 잇따라 잇단 철도사고에 국...,NaN,"전날, 선로, 전기, 공급, 차단, 사고, 여파, 이틀, 운행, 취소, 연기, 철도..."
4,AKR20221231025100003,선로 전기공급 끊기고 SRT 고장이틀째 대거 운행지연 취소,첫날 KTX SRT 최대 2시간여 운행지연열차 운행 축소 터널 하자보수 공사때 쓴 ...,NaN,"날, 최대, 운행, 지연, 열차, 운행, 축소, 터널, 하자, 보수, 공사, 때, ..."
...,...,...,...,...,...
14052,AKR20220607103300530,"소방청 가스안전공사, 신에너지 사고 대응 업무협약",계승현 기자 = 소방청은 8일 한국가스안전공사와 수소 등 신에너지 관련 사고에 대응...,NaN,"계승, 현, 기자, 소방, 청, 수소, 에너지, 관련, 사고, 대응, 업무, 협약,..."
14053,PYH20220608015200017,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축",LG유플러스가 서울 지하철 8호선 18개 역사에 '스마트스테이션'을 구축하는 사업을...,NaN,"지하철, 역사, 스마트, 스테이션, 구축, 사업, 성공, 스마트, 스테이션, 지하철..."
14054,AKR20220607168000017,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축 완료",역사 내 사고에 신속 대응해 역무 효율성 높여 임성호 기자 = LG유플러스는 서울 ...,NaN,"역사, 사고, 신속, 대응, 역무, 효율, 기자, 지하철, 역사, 스마트, 스테이션..."
14055,AKR20220608017300530,제조업 운반 하역 작업서 부상 가장 많이 발생전체 19.3%,김승욱 기자 = 3일 이상의 휴업이 필요한 부상 사고의 20% 가까이가 제조업 운반...,NaN,"기자, 이상, 휴업, 필요, 부상, 사고, 가까이, 제조업, 운반, 하역, 작업, ..."


In [37]:
jac =[]
for i in range(len(df_cluster_KI)):
        for j in range(len(df_cluster)):
                jac.append([df_cluster_KI['Title'][i],df_cluster['Title'][j],jaccard_similarity(df_cluster_KI['NNG'][i],df_cluster['NNG'][j])])

In [38]:
jac = pd.DataFrame(jac).sort_values(2,ascending=False).drop_duplicates(subset=1)

In [39]:
jac[jac[2] > 0.043]

,0,1,2
14169,과천 제2경인고속도 방음터널서 화재5명 사망 37명 부상,과천 제2경인고속도 방음터널서 화재5명 사망 37명 부상,1.000000
92,과천 제2경인고속도 방음터널서 큰 불5명 사망 37명 부상,과천 제2경인고속도 방음터널서 큰 불5명 사망 37명 부상,1.000000
14180,과천 제2경인고속도 방음터널서 화재5명 사망 37명 부상,제2경인고속도 방음터널서 화재6명 사망 29명 부상,0.838926
14155,과천 제2경인고속도 방음터널서 화재5명 사망 37명 부상,폭격 맞은 듯한 과천 방음터널골조만 남은채 매캐한 냄새 진동,0.617647
14190,과천 제2경인고속도 방음터널서 화재5명 사망 37명 부상,제2경인고속도로 북의왕IC 부근 방음터널서 화재2명 사망,0.615894
...,...,...,...
17055,과천 제2경인고속도 방음터널서 화재5명 사망 37명 부상,최고위원회의에서 발언하는 이재명 대표,0.046358
25612,과천 제2경인고속도 방음터널서 화재5명 사망 37명 부상,건설현장 등 큰 피해는 없어현장상황 점검 분주,0.045752
24878,과천 제2경인고속도 방음터널서 화재5명 사망 37명 부상,폭우로 무너진 세종시 주택 축대,0.045161
8477,과천 제2경인고속도 방음터널서 큰 불5명 사망 37명 부상,중대본 태풍으로 1명 사망 2명 실종,0.043269


In [40]:
jac_ITW =[]
for i in range(len(df_cluster_ITW)):
        for j in range(len(df_cluster_ITW)):
                jac_ITW.append([df_cluster_ITW['Title'][i],df_cluster['Title'][j],jaccard_similarity(df_cluster_ITW['NNG'][i],df_cluster['NNG'][j])])

NameError: name 'df_cluster_ITW' is not defined

In [ ]:
cond1 = 1 > pd.DataFrame(jac_ITW)[2] 

In [ ]:
pd.DataFrame(jac_ITW).loc[cond1].sort_values(by=2,ascending=False).head(50)

In [ ]:
df_cluster_ITW

In [ ]:
df_cluster.dropna()['li'] == '중부 집중호우'

In [ ]:
df_cluster_3 = df_cluster[df_cluster['li'] == '중부 집중호우'].reset_index().drop('index',axis=1)

In [ ]:
df_cluster_3

In [ ]:
jac =[]
for i in range(len(df_cluster_3)):
        for j in range(len(df_cluster_3)):
                jac.append([df_cluster_3['Body'][i],df_cluster['Title'][j],jaccard_similarity(df_cluster_3['NNG'][i],df_cluster['NNG'][j])])

In [ ]:
cond1 = 1 > pd.DataFrame(jac)[2] 

In [ ]:
pd.DataFrame(jac).loc[cond1].sort_values(by=2,ascending=False).head(50)

In [ ]:
df_cluster_KI

In [ ]:
from kiwipiepy import Kiwi
kiwi = Kiwi()
b = pd.DataFrame()

for i in df['Body']:
    a = []
    c = ''
    for result, score in kiwi.analyze(i, top_n=1):
        for j in range(len(result)):
            if result[j][1] == 'NNG':
                c += result[j][0] + ', '
    a.append(c)
    b = pd.concat([b,pd.DataFrame([a])],axis=0)

In [41]:
df

,_id,send_timestamp,Title,Body,YNewsML.Metadata.Region.RegionCode,Summarization,NamedEntity,Keyword,img_link,issue_word,reason,class_code,class_name
0,AKR20221231041400061,2022-12-31T23:20:59,신분당선 판교∼정자역 사이서 전동차 멈춰1시간가량 불편,이영주 기자 = 31일 오후 9시 40분께 경기도 성남시 신분당선 판교역과 정자역 ...,"[{'code': 7005013, 'name': '성남'}, {'code': 700...",31일 오후 9시 40분께 경기도 성남시 신분당선 판교역과 정자역 사이 선로에서 전...,"[{'code': 'AF', 'word': '신분당선'}, {'code': 'LC'...","['정자역', '신분당선', '선로', '고장', '구원열차', '탑승', '꺼졌'...",img5.yna.co.kr/photo/cms/2016/05/02/01/C0A8CA3...,NaN,NaN,606010.0,철도사고
1,AKR20221231035000099,2022-12-31T18:08:13,남아공 가스트럭 폭발사고 사망자 34명으로 늘어,김성진 특파원 = 남아프리카공화국에서 지난 크리스마스이브에 발생한 가스 트럭 폭발 ...,NaN,"남아프리카공화국에서 지난 크리스마스이브에 발생한 가스 트럭 폭발 사고와 관련, 사망...","[{'code': 'LC', 'word': '남아공'}, {'code': 'AF',...","['남아공', '사망자', '폭발', '환자']",img8.yna.co.kr/photo/reuters/2022/12/29/PRU202...,NaN,NaN,606012.0,폭발사고
2,AKR20221231025151003,2022-12-31T16:16:20,"선로에 떨어진 부직포 때문에SRT, 운행 차질에 차량 고장까지",전날 선로 전기공급 차단 사고 여파이틀째 운행 취소 연기 잇따라 잇단 철도사고에 국...,NaN,선로 전기공급 차단 문제로 수서고속철도(SRT) 열차 운행이 이틀째 차질을 빚고 있...,"[{'code': 'AF', 'word': 'SRT'}, {'code': 'AF',...","['열차', '지연', 'GS건설', '고장', '전기공급', '차단', '통복터널']",img3.yna.co.kr/photo/yna/YH/2022/12/20/PYH2022...,NaN,NaN,606010.0,철도사고
3,AKR20221231025151003,2022-12-31T16:16:20,"선로에 떨어진 부직포 때문에SRT, 운행 차질에 차량 고장까지",전날 선로 전기공급 차단 사고 여파이틀째 운행 취소 연기 잇따라 잇단 철도사고에 국...,NaN,선로 전기공급 차단 문제로 수서고속철도(SRT) 열차 운행이 이틀째 차질을 빚고 있...,"[{'code': 'AF', 'word': 'SRT'}, {'code': 'AF',...","['열차', '지연', 'GS건설', '고장', '전기공급', '차단', '통복터널']",img3.yna.co.kr/photo/yna/YH/2022/12/20/PYH2022...,NaN,NaN,606001.0,사고일반
4,AKR20221231025100003,2022-12-31T12:57:34,선로 전기공급 끊기고 SRT 고장이틀째 대거 운행지연 취소,첫날 KTX SRT 최대 2시간여 운행지연열차 운행 축소 터널 하자보수 공사때 쓴 ...,NaN,선로 전기공급 차단 문제로 수서고속철도(SRT) 열차 운행이 이틀째 차질을 빚고 있...,"[{'code': 'AF', 'word': 'SRT'}, {'code': 'DT',...","['열차', '지연', 'SR', '전기공급', '끊기', '통복터널']",img4.yna.co.kr/photo/yna/YH/2022/12/20/PYH2022...,NaN,NaN,606010.0,철도사고
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14052,AKR20220607103300530,2022-06-08T10:00:07,"소방청 가스안전공사, 신에너지 사고 대응 업무협약",계승현 기자 = 소방청은 8일 한국가스안전공사와 수소 등 신에너지 관련 사고에 대응...,NaN,소방청은 8일 한국가스안전공사와 수소 등 신에너지 관련 사고에 대응하기 위해 업무협...,"[{'code': 'OG', 'word': '소방청가스안전공사'}, {'code':...","['한국가스안전공사', '사고수습', '수소경제', '기술교육', '소방청', '대...",img5.yna.co.kr/photo/cms/2019/04/16/62/PCM2019...,NaN,NaN,606008.0,안전관리
14053,PYH20220608015200017,2022-06-08T09:20:49,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축",LG유플러스가 서울 지하철 8호선 18개 역사에 '스마트스테이션'을 구축하는 사업을...,NaN,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축 LG유플러스가 서울 지하철 8호...","[{'code': 'OG', 'word': 'LGU+'}, {'code': 'AF'...","['lgu', '구축', '8호선', 'cctv', '스마트스테이션', '고도', ...",img4.yna.co.kr/photo/yna/YH/2022/06/07/PYH2022...,NaN,NaN,606008.0,안전관리
14054,AKR20220607168000017,2022-06-08T09:00:02,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축 완료",역사 내 사고에 신속 대응해 역무 효율성 높여 임성호 기자 = LG유플러스는 서울 ...,NaN,LG유플러스는 서울 지하철 8호선 18개 역사에 '스마트스테이션'을 구축하는 사업을...,"[{'code': 'OG', 'word': 'LGU+'}, {'code': 'AF'...","['lg유플러스하이트', '서울교통공사', '역무원', '8호선', '스마트스테이션...",img9.yna.co.kr/etc/inner/KR/2022/06/07/AKR2022...,NaN,NaN,606008.0,안전관리
14055,AKR20220608017300530,2022-06-08T09:00:00,제조업 운반 하역 작업서 부상 가장 많이 발생전체 19.3%,김승욱 기자 = 3일 이상의 휴업이 필요한 부상 사고의 20% 가까이가 제조업 운반...,NaN,3일 이상의 휴업이 필요한 부상 사고의 20% 가까이가 제조업 운반·하역 작업에서 ...,"[{'code': 'DT', 'word': '3일 이상'}, {'code': 'QT...","['산업재해', '작업', '지방고용노동관서', '휴업', '3년간']",img7.yna.co.kr/photo/cms/2018/12/13/00/PCM2018...,NaN,NaN,606006.0,산재


In [63]:
df['issue_word_1'] = np.nan

In [69]:
for i in range(len(df)):
    if len(df['issue_word'].fillna("0")[i]) > 2:
            df['issue_word_1'][i] = df['issue_word'][i]
    else:
        pass

<ipython-input-69-ec9b4b1306e6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['issue_word_1'][i] = df['issue_word'][i]


In [81]:
df['issue_word_1'] = df['issue_word_1'].replace('10.29 참사','이태원압사참사')

In [76]:
df['issue_word_1'] = df['issue_word_1'].replace('10·29참사','이태원압사참사')

In [83]:
issue_list = df['issue_word_1'].value_counts().index.tolist()

In [96]:
issue_list_space = []
for i in issue_list:
   issue_list_space.append( kiwi.space(i,reset_whitespace=True))

In [97]:
issue_list_space

['이태원 압사 참사',
 '힌남노 태풍',
 '핼러윈',
 '폭우 산사태',
 '풍수해',
 '학동 참사',
 '태풍 난마 돌이',
 '재난 안전 대응 훈련',
 '화정 아이파크',
 '4.9',
 '을지 태극 연습',
 '수능 시험',
 '세월호 침몰 사고',
 '정기 국회',
 '대한민국',
 '월드컵',
 '중앙 지방 정책 협의회',
 '봉화 광산 매몰 사고',
 '소방 기술 경연 대회',
 '산업 안전 보건 강조',
 '쉐이크 아웃(ShakeOut) 코리아',
 '10·29 참사',
 'SPL 사고',
 '안전 문화 포럼',
 '코로나 19',
 '허리케인',
 '우루과이 전',
 '국민 안전 발명 챌린지',
 '하 이선',
 '후쿠시마',
 '제야의 종 타종 행사',
 '태풍무이파',
 '스텔라데이지호 사고',
 '우크라이나 전쟁',
 '에어리',
 '부산불꽃축제',
 '지방 선거',
 '안 전 보건 관리 체계 구축 컨설팅 중소기업 우수 사례 발표 대회',
 '막국수 닭갈비 축제',
 '88 올림픽',
 'Bon Om Touk',
 '조별 리그',
 '부산 세계 박람회 유치 기원 콘서트',
 'SRF 설명회',
 '긴급 구조 종합 훈련',
 '가을 문화 축제',
 '창덕궁 달빛 기행',
 '안 전한국 훈련',
 '체르노빌',
 '해맞이 축제',
 '카니발',
 '동일본 대지진',
 '촛불 집회',
 '고인돌 전국 마라톤 대회',
 '삼풍백화점 사건',
 '재난 안전 콘퍼런스',
 '송끄란 축제',
 '대테러 종합 훈련',
 "태풍 '차바",
 'Vigilant Storm',
 '항공기 사고 대응 모의 훈련',
 '여의도 불꽃 축제',
 '연평도 포격 도발',
 '이륜 차가 안전한 대한민국을 위한 정책 세미나',
 '빨래 골 축제',
 '통신 서비스 이용자 주간',
 '경기도 생활 대축전',
 '산업 재해 노동자',
 'DXKOREA',
 '남양주 개 물림 사망 사건',
 '로이끄라 통 축제',
 '조계사',
 '빅 스마일 데이',
 '블랙아웃',
 

In [44]:
df

,_id,send_timestamp,Title,Body,YNewsML.Metadata.Region.RegionCode,Summarization,NamedEntity,Keyword,img_link,issue_word,reason,class_code,class_name
0,AKR20221231041400061,2022-12-31T23:20:59,신분당선 판교∼정자역 사이서 전동차 멈춰1시간가량 불편,이영주 기자 = 31일 오후 9시 40분께 경기도 성남시 신분당선 판교역과 정자역 ...,"[{'code': 7005013, 'name': '성남'}, {'code': 700...",31일 오후 9시 40분께 경기도 성남시 신분당선 판교역과 정자역 사이 선로에서 전...,"[{'code': 'AF', 'word': '신분당선'}, {'code': 'LC'...","['정자역', '신분당선', '선로', '고장', '구원열차', '탑승', '꺼졌'...",img5.yna.co.kr/photo/cms/2016/05/02/01/C0A8CA3...,NaN,NaN,606010.0,철도사고
1,AKR20221231035000099,2022-12-31T18:08:13,남아공 가스트럭 폭발사고 사망자 34명으로 늘어,김성진 특파원 = 남아프리카공화국에서 지난 크리스마스이브에 발생한 가스 트럭 폭발 ...,NaN,"남아프리카공화국에서 지난 크리스마스이브에 발생한 가스 트럭 폭발 사고와 관련, 사망...","[{'code': 'LC', 'word': '남아공'}, {'code': 'AF',...","['남아공', '사망자', '폭발', '환자']",img8.yna.co.kr/photo/reuters/2022/12/29/PRU202...,NaN,NaN,606012.0,폭발사고
2,AKR20221231025151003,2022-12-31T16:16:20,"선로에 떨어진 부직포 때문에SRT, 운행 차질에 차량 고장까지",전날 선로 전기공급 차단 사고 여파이틀째 운행 취소 연기 잇따라 잇단 철도사고에 국...,NaN,선로 전기공급 차단 문제로 수서고속철도(SRT) 열차 운행이 이틀째 차질을 빚고 있...,"[{'code': 'AF', 'word': 'SRT'}, {'code': 'AF',...","['열차', '지연', 'GS건설', '고장', '전기공급', '차단', '통복터널']",img3.yna.co.kr/photo/yna/YH/2022/12/20/PYH2022...,NaN,NaN,606010.0,철도사고
3,AKR20221231025151003,2022-12-31T16:16:20,"선로에 떨어진 부직포 때문에SRT, 운행 차질에 차량 고장까지",전날 선로 전기공급 차단 사고 여파이틀째 운행 취소 연기 잇따라 잇단 철도사고에 국...,NaN,선로 전기공급 차단 문제로 수서고속철도(SRT) 열차 운행이 이틀째 차질을 빚고 있...,"[{'code': 'AF', 'word': 'SRT'}, {'code': 'AF',...","['열차', '지연', 'GS건설', '고장', '전기공급', '차단', '통복터널']",img3.yna.co.kr/photo/yna/YH/2022/12/20/PYH2022...,NaN,NaN,606001.0,사고일반
4,AKR20221231025100003,2022-12-31T12:57:34,선로 전기공급 끊기고 SRT 고장이틀째 대거 운행지연 취소,첫날 KTX SRT 최대 2시간여 운행지연열차 운행 축소 터널 하자보수 공사때 쓴 ...,NaN,선로 전기공급 차단 문제로 수서고속철도(SRT) 열차 운행이 이틀째 차질을 빚고 있...,"[{'code': 'AF', 'word': 'SRT'}, {'code': 'DT',...","['열차', '지연', 'SR', '전기공급', '끊기', '통복터널']",img4.yna.co.kr/photo/yna/YH/2022/12/20/PYH2022...,NaN,NaN,606010.0,철도사고
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14052,AKR20220607103300530,2022-06-08T10:00:07,"소방청 가스안전공사, 신에너지 사고 대응 업무협약",계승현 기자 = 소방청은 8일 한국가스안전공사와 수소 등 신에너지 관련 사고에 대응...,NaN,소방청은 8일 한국가스안전공사와 수소 등 신에너지 관련 사고에 대응하기 위해 업무협...,"[{'code': 'OG', 'word': '소방청가스안전공사'}, {'code':...","['한국가스안전공사', '사고수습', '수소경제', '기술교육', '소방청', '대...",img5.yna.co.kr/photo/cms/2019/04/16/62/PCM2019...,NaN,NaN,606008.0,안전관리
14053,PYH20220608015200017,2022-06-08T09:20:49,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축",LG유플러스가 서울 지하철 8호선 18개 역사에 '스마트스테이션'을 구축하는 사업을...,NaN,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축 LG유플러스가 서울 지하철 8호...","[{'code': 'OG', 'word': 'LGU+'}, {'code': 'AF'...","['lgu', '구축', '8호선', 'cctv', '스마트스테이션', '고도', ...",img4.yna.co.kr/photo/yna/YH/2022/06/07/PYH2022...,NaN,NaN,606008.0,안전관리
14054,AKR20220607168000017,2022-06-08T09:00:02,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축 완료",역사 내 사고에 신속 대응해 역무 효율성 높여 임성호 기자 = LG유플러스는 서울 ...,NaN,LG유플러스는 서울 지하철 8호선 18개 역사에 '스마트스테이션'을 구축하는 사업을...,"[{'code': 'OG', 'word': 'LGU+'}, {'code': 'AF'...","['lg유플러스하이트', '서울교통공사', '역무원', '8호선', '스마트스테이션...",img9.yna.co.kr/etc/inner/KR/2022/06/07/AKR2022...,NaN,NaN,606008.0,안전관리
14055,AKR20220608017300530,2022-06-08T09:00:00,제조업 운반 하역 작업서 부상 가장 많이 발생전체 19.3%,김승욱 기자 = 3일 이상의 휴업이 필요한 부상 사고의 20% 가까이가 제조업 운반...,NaN,3일 이상의 휴업이 필요한 부상 사고의 20% 가까이가 제조업 운반·하역 작업에서 ...,"[{'code': 'DT', 'word': '3일 이상'}, {'code': 'QT...","['산업재해', '작업', '지방고용노동관서', '휴업', '3년간']",img7.yna.co.kr/photo/cms/2018/12/13/00/PCM2018...,NaN,NaN,606006.0,산재


In [99]:
df['issue_list'] = np.nan

In [100]:
for i in issue_list_space:
    for j in range(len(df['Body'])):
        if i in df['Body'][j]:
                df['issue_list'][j] = i

<ipython-input-100-4e8a822efdf2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['issue_list'][j] = i


In [106]:
df

,_id,send_timestamp,Title,Body,YNewsML.Metadata.Region.RegionCode,Summarization,NamedEntity,Keyword,img_link,issue_word,reason,class_code,class_name,issue_list,issue_word_1
0,AKR20221231041400061,2022-12-31T23:20:59,신분당선 판교∼정자역 사이서 전동차 멈춰1시간가량 불편,이영주 기자 = 31일 오후 9시 40분께 경기도 성남시 신분당선 판교역과 정자역 ...,"[{'code': 7005013, 'name': '성남'}, {'code': 700...",31일 오후 9시 40분께 경기도 성남시 신분당선 판교역과 정자역 사이 선로에서 전...,"[{'code': 'AF', 'word': '신분당선'}, {'code': 'LC'...","['정자역', '신분당선', '선로', '고장', '구원열차', '탑승', '꺼졌'...",img5.yna.co.kr/photo/cms/2016/05/02/01/C0A8CA3...,NaN,NaN,606010.0,철도사고,NaN,NaN
1,AKR20221231035000099,2022-12-31T18:08:13,남아공 가스트럭 폭발사고 사망자 34명으로 늘어,김성진 특파원 = 남아프리카공화국에서 지난 크리스마스이브에 발생한 가스 트럭 폭발 ...,NaN,"남아프리카공화국에서 지난 크리스마스이브에 발생한 가스 트럭 폭발 사고와 관련, 사망...","[{'code': 'LC', 'word': '남아공'}, {'code': 'AF',...","['남아공', '사망자', '폭발', '환자']",img8.yna.co.kr/photo/reuters/2022/12/29/PRU202...,NaN,NaN,606012.0,폭발사고,NaN,NaN
2,AKR20221231025151003,2022-12-31T16:16:20,"선로에 떨어진 부직포 때문에SRT, 운행 차질에 차량 고장까지",전날 선로 전기공급 차단 사고 여파이틀째 운행 취소 연기 잇따라 잇단 철도사고에 국...,NaN,선로 전기공급 차단 문제로 수서고속철도(SRT) 열차 운행이 이틀째 차질을 빚고 있...,"[{'code': 'AF', 'word': 'SRT'}, {'code': 'AF',...","['열차', '지연', 'GS건설', '고장', '전기공급', '차단', '통복터널']",img3.yna.co.kr/photo/yna/YH/2022/12/20/PYH2022...,NaN,NaN,606010.0,철도사고,오봉역,NaN
3,AKR20221231025151003,2022-12-31T16:16:20,"선로에 떨어진 부직포 때문에SRT, 운행 차질에 차량 고장까지",전날 선로 전기공급 차단 사고 여파이틀째 운행 취소 연기 잇따라 잇단 철도사고에 국...,NaN,선로 전기공급 차단 문제로 수서고속철도(SRT) 열차 운행이 이틀째 차질을 빚고 있...,"[{'code': 'AF', 'word': 'SRT'}, {'code': 'AF',...","['열차', '지연', 'GS건설', '고장', '전기공급', '차단', '통복터널']",img3.yna.co.kr/photo/yna/YH/2022/12/20/PYH2022...,NaN,NaN,606001.0,사고일반,오봉역,NaN
4,AKR20221231025100003,2022-12-31T12:57:34,선로 전기공급 끊기고 SRT 고장이틀째 대거 운행지연 취소,첫날 KTX SRT 최대 2시간여 운행지연열차 운행 축소 터널 하자보수 공사때 쓴 ...,NaN,선로 전기공급 차단 문제로 수서고속철도(SRT) 열차 운행이 이틀째 차질을 빚고 있...,"[{'code': 'AF', 'word': 'SRT'}, {'code': 'DT',...","['열차', '지연', 'SR', '전기공급', '끊기', '통복터널']",img4.yna.co.kr/photo/yna/YH/2022/12/20/PYH2022...,NaN,NaN,606010.0,철도사고,오봉역,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14052,AKR20220607103300530,2022-06-08T10:00:07,"소방청 가스안전공사, 신에너지 사고 대응 업무협약",계승현 기자 = 소방청은 8일 한국가스안전공사와 수소 등 신에너지 관련 사고에 대응...,NaN,소방청은 8일 한국가스안전공사와 수소 등 신에너지 관련 사고에 대응하기 위해 업무협...,"[{'code': 'OG', 'word': '소방청가스안전공사'}, {'code':...","['한국가스안전공사', '사고수습', '수소경제', '기술교육', '소방청', '대...",img5.yna.co.kr/photo/cms/2019/04/16/62/PCM2019...,NaN,NaN,606008.0,안전관리,NaN,NaN
14053,PYH20220608015200017,2022-06-08T09:20:49,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축",LG유플러스가 서울 지하철 8호선 18개 역사에 '스마트스테이션'을 구축하는 사업을...,NaN,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축 LG유플러스가 서울 지하철 8호...","[{'code': 'OG', 'word': 'LGU+'}, {'code': 'AF'...","['lgu', '구축', '8호선', 'cctv', '스마트스테이션', '고도', ...",img4.yna.co.kr/photo/yna/YH/2022/06/07/PYH2022...,NaN,NaN,606008.0,안전관리,NaN,NaN
14054,AKR20220607168000017,2022-06-08T09:00:02,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축 완료",역사 내 사고에 신속 대응해 역무 효율성 높여 임성호 기자 = LG유플러스는 서울 ...,NaN,LG유플러스는 서울 지하철 8호선 18개 역사에 '스마트스테이션'을 구축하는 사업을...,"[{'code': 'OG', 'word': 'LGU+'}, {'code': 'AF'...","['lg유플러스하이트', '서울교통공사', '역무원', '8호선', '스마트스테이션...",img9.yna.co.kr/etc/inner/KR/2022/06/07/AKR2022...,NaN,NaN,606008.0,안전관리,NaN,NaN
14055,AKR20220608017300530,2022-06-08T09:00:00,제조업 운반 하역 작업서 부상 가장 많이 발생전체 19.3%,김승욱 기자 = 3일 이상의 휴업이 필요한 부상 사고의 20% 가까이가 제조업 운반...,NaN,3일 이상의 휴업이 필요한 부상 사고의 20% 가까이가 제조업 운반·하역 작업에서 ...,"[{'code': 'DT', 'word': '3일 이상'}, {'code': 'QT...","['산업재해', '작업', '지방고용노동관서', '휴업', '3년간']",img7.yna.co.kr/photo/cms/2018/12/13/00/PCM2018...,NaN,NaN,606006.0,산재,NaN,NaN


In [111]:
df_news[0][0]

'특파원, 발생, 북서부, 카지노, 호텔, 대, 화재, 사망자, 최소, 통신, 외신, 이번, 화재, 사망자, 집계, 가운데, 구조, 수색, 작업, 종료, 실종, 상태, 치료, 부상자, 일부, 중태, 실제, 희생자, 사망자, 다수, 태국인, 국경, 카지노, 호텔, 태국인, 고객, 직원, 카지노, 불법, 오후, 접경, 다이아몬드, 시티, 호텔, 카지노, 큰불, 불, 순식, 건물, 전체, 대피, 창문, 참사, 정오, 진화, 이후, 구조대, 내부, 수색, 작업, 총리, 희생자, 유가족, 애도, 화재, 예방, 대응, 강화, 약속, '

In [43]:
df2

,issue_word,NNG
0,10,"주최, 운집, 예상, 경기, 최대, 대비, 안전, 관리, 요원, 배치, 도민, 응원..."
1,10.29 참사,"교육, 교육, 강화, 예산, 증액, 기자, 일반, 시행, 과거, 주요국, 수준, 국..."
2,2022 쉐이크아웃 코리아,"기자, 오전, 희망, 브리지, 지진, 대비, 훈련, 쉐이크, 아웃, 비상, 사용, ..."
3,2차전,"인파, 관리, 질서, 유지, 대테러, 담당, 규모, 추후, 결정, 기자, 경찰, 월..."
4,3대,"농구, 연습, 누전, 조명, 탑, 접촉, 농구, 애도, 특파원, 활동, 러시아인, ..."
...,...,...
231,화재,"특파원, 발생, 북서부, 카지노, 호텔, 대, 화재, 사망자, 최소, 통신, 외신,..."
232,화정아이파크,"책임자, 비위, 행위, 추가, 책임, 소재, 법정, 다툼, 주목, 기자, 붕괴, 사..."
233,후쿠시마,"운전, 정비, 비상, 대응, 분야, 중점, 점검, 원, 기자, 안전, 점검, 팀, ..."
234,히잡 의문사' 사건,"당국, 범죄자, 과정, 불, 정부, 시위, 무관, 정치범, 수감, 인권, 처우, 악..."
